<a href="https://colab.research.google.com/github/tejaschaudhari2811/IMDB_Sentiment_Classification_Pure_Tensorflow/blob/main/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd


In [2]:
imdb_train, df_info = tfds.load(name = "imdb_reviews", split='train', with_info=True, as_supervised=True)
imdb_test = tfds.load(name = "imdb_reviews", split='test', as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBBHO6C/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBBHO6C/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
print(df_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [4]:
for example,label in imdb_train.take(1):
  print(example, "\n", label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [5]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS = 0
for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
    MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)  

In [6]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, 
                                                    tokenizer = tokenizer)
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [7]:
for example,label in imdb_train.take(1):
  print(example)
  encoded = imdb_encoder.encode(example.numpy())
  print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [8]:
imdb_encoder.save_to_file('reviews_vocab')

In [9]:
enc = tfds.deprecated.text.TokenTextEncoder.load_from_file('reviews_vocab')
enc.decode(enc.encode('Good Case, Excellent Value.'))  

'Good Case Excellent Value'

In [10]:
from tensorflow.keras.preprocessing import sequence
def encode_pad_transform(sample):
  encoded = imdb_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded], padding = "post",
                               maxlen=150)
  return np.array(pad[0])

def encode_tf_fn(sample, label):
  encoded = tf.py_function(encode_pad_transform, inp=[sample], 
                           Tout = (tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label

In [11]:
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)
for review, label in tst.take(1):
  print(review, label)
  print(imdb_encoder.decode(review))


tf.Tensor(
[62998  3420 86553 69140 25557 41705 59943  2645 78194 23207 53021 35257
 78787  6636 41292 83171 26478 27380  5558 50660 89213 92897 10875 28296
 73934 78194 88486 81720 54179 53021 69854 62438 88486 50660 77873 36006
 62604 12891 10875 41705 55294 13076 36890 62998 41705 43437 86553 36526
 45946 38207 60461 13486 41414 20987 20261 52369 67583 29528 92522 57666
 40932 76982 67583 44345 88486 79507 53257 36970 24941 20980 71189 15781
 16308 38233 85399 70382 61288  5275 75163  6636  3420 36379 92897 10162
 20261 57713 58765 53021 10162 41705 81651  3420 72816 85031 19603 46573
 81647 57643 11782 68108 81651 14586  5558 21180   691 10875 16870 93698
 55294   691 78787  6636 55294 11605 30011 57643 36006 81228 30276 79584
 48025     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [12]:
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [13]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         mask_zero=True, batch_input_shape=
                                                         [batch_size, None]),
          tf.keras.layers.LSTM(rnn_units),
          tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [14]:
# Parameters Configuration
vocab_size = imdb_encoder.vocab_size

embedding_dim = 64

rnn_units = 64

batch_size  = 100

In [15]:
model = build_model_lstm(vocab_size= vocab_size, embedding_dim= embedding_dim,
                         rnn_units = rnn_units, batch_size = batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall'])
encoded_train_batched = encoded_train.batch(batch_size)
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 49s 159ms/step - loss: 0.4174 - accuracy: 0.7981 - precision: 0.7782 - recall: 0.8338
Epoch 2/10
250/250 [==============================] - 40s 159ms/step - loss: 0.1710 - accuracy: 0.9382 - precision: 0.9394 - recall: 0.9368
Epoch 3/10
250/250 [==============================] - 40s 158ms/step - loss: 0.1039 - accuracy: 0.9655 - precision: 0.9680 - recall: 0.9629
Epoch 4/10
250/250 [==============================] - 40s 158ms/step - loss: 0.0908 - accuracy: 0.9685 - precision: 0.9683 - recall: 0.9686
Epoch 5/10
250/250 [==============================] - 39s 157ms/step - loss: 0.0601 - accuracy: 0.9795 - precision: 0.9801 - recall: 0.9789
Epoch 6/10
250/250 [==============================] - 39s 157ms/step - loss: 0.0483 - accuracy: 0.9848 - precision: 0.9859 - recall: 0.9837
Epoch 7/10
250/250 [==============================] - 39s 157ms/step - loss: 0.0309 - accuracy: 0.9903 - precision: 0.9903 - recall: 0.9903
Epoch 8/10
250/250 [

In [18]:
model.evaluate(encoded_test.batch(batch_size))

250/250 [==============================] - 34s 131ms/step - loss: 0.6686 - accuracy: 0.8314 - precision: 0.8190 - recall: 0.8507


[0.6686300039291382,
 0.8313599824905396,
 0.8190079927444458,
 0.8507199883460999]

In [25]:
# Converting model to Bidirectional
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         mask_zero=True, batch_input_shape=
                                                         [batch_size, None]),
          tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units)),
          tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [26]:
bilstm = build_model_bilstm(vocab_size= vocab_size, embedding_dim= embedding_dim,
                         rnn_units = rnn_units, batch_size = batch_size)
bilstm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (100, None, 64)           6011584   
_________________________________________________________________
bidirectional_4 (Bidirection (100, 128)                66048     
_________________________________________________________________
dense_5 (Dense)              (100, 1)                  129       
Total params: 6,077,761
Trainable params: 6,077,761
Non-trainable params: 0
_________________________________________________________________


In [29]:
bilstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall'])
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
250/250 [==============================] - 57s 197ms/step - loss: 0.3670 - accuracy: 0.8456 - precision: 0.8414 - recall: 0.8517
Epoch 2/5
250/250 [==============================] - 49s 198ms/step - loss: 0.2087 - accuracy: 0.9249 - precision: 0.9203 - recall: 0.9303
Epoch 3/5
250/250 [==============================] - 49s 197ms/step - loss: 0.0960 - accuracy: 0.9693 - precision: 0.9685 - recall: 0.9702
Epoch 4/5
250/250 [==============================] - 50s 198ms/step - loss: 0.0591 - accuracy: 0.9811 - precision: 0.9792 - recall: 0.9831
Epoch 5/5
250/250 [==============================] - 49s 198ms/step - loss: 0.0392 - accuracy: 0.9883 - precision: 0.9879 - recall: 0.9887


In [30]:
# Model prediction for movie "Shang-Chi and the Legend of the Ten Rings" 
# Actual Sentiment :- Positive

Sample = "It was good. So far, it's in my top10 list of 2021 movies. However, I wasn't rly hooked up, jokes seemed to be outdated, some scenes were rushed. I liked visuals but that's it, I expected more from Marvel."

In [32]:
sample = tf.convert_to_tensor(Sample)

In [38]:
sample_encoded= encode_pad_transform(sample)